In [1]:
# This notebook checks is trying different pipeline approaches

# Cropped data is used here (5 minutes only), tried on whole data - takes forever.


#Load data, make folders
import numpy as np
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import mne
import pyprep as pp


In [2]:

#from Functions.main_meg_qc import initial_stuff
from Functions. data_load_and_folders import load_meg_data
duration=10
#n_events, df_epochs_mags, df_epochs_grads, epochs_mags, epochs_grads, mags, grads, filtered_d, filtered_d_resamp, raw_cropped, raw=initial_stuff(duration)

raw_file = './data/sub_HT05ND16/210811/mikado-1.fif/'
raw, mags, grads = load_meg_data(raw_file)

#crop the data to calculate faster
raw_cropped = raw.copy()
raw_cropped.crop(0, duration*60) 



Opening raw data file ./data/sub_HT05ND16/210811/mikado-1.fif/...
    Read a total of 8 projection items:
        magn8_iasoff_68deg.fif : PCA-v1 (1 x 306)  idle
        magn8_iasoff_68deg.fif : PCA-v2 (1 x 306)  idle
        magn8_iasoff_68deg.fif : PCA-v3 (1 x 306)  idle
        magn8_iasoff_68deg.fif : PCA-v4 (1 x 306)  idle
        magn8_iasoff_68deg.fif : PCA-v5 (1 x 306)  idle
        magn8_iasoff_68deg.fif : PCA-v6 (1 x 306)  idle
        magn8_iasoff_68deg.fif : PCA-v7 (1 x 306)  idle
        magn8_iasoff_68deg.fif : PCA-v8 (1 x 306)  idle
    Range : 1809000 ... 3375999 =   1809.000 ...  3375.999 secs
Ready.
Opening raw data file /Users/jenya/Local Storage/Job Uni Rieger lab/MEG QC code/data/sub_HT05ND16/210811/mikado-2.fif...
    Read a total of 8 projection items:
        magn8_iasoff_68deg.fif : PCA-v1 (1 x 306)  idle
        magn8_iasoff_68deg.fif : PCA-v2 (1 x 306)  idle
        magn8_iasoff_68deg.fif : PCA-v3 (1 x 306)  idle
        magn8_iasoff_68deg.fif : PCA-v4 (1 x 3

/Users/jenya/Local Storage/Job Uni Rieger lab/MEG QC code/Functions/data_load_and_folders.py:8: RuntimeWarning: This filename (./data/sub_HT05ND16/210811/mikado-1.fif/) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  raw = mne.io.read_raw_fif(raw_file)


<Raw | mikado-1.fif, 319 x 600001 (600.0 s), ~6.6 MB, data not loaded>

In [3]:
# Annotate bad peak to peak amplitudes: peaks and flats

# https://mne.tools/stable/generated/mne.preprocessing.annotate_amplitude.html
# copmpare this MNE function with what I wrote myself: Funnks - > Peaks_meg_qc -> neighbour_peak_amplitude

#'Creates annotations BAD_peak or BAD_flat for spans of data where consecutive samples exceed the threshold in peak or fall below the threshold 
# in flat for more than min_duration' 

amplit_annot=mne.preprocessing.annotate_amplitude(raw_cropped, peak={'mag':3.5e-11, 'grad': 4e-11}, flat=3e-15, bad_percent=5, min_duration=0.005, verbose=True)

# Some input parameters:
# * bad_percent - percent of tolerated bad segments in the channel. id some channel is over this percent - will be returned as "bads". 
#   While if its under, just the points will be returned as "bad peak or "bad flat"
#   However I dont see any info about which channel these points belong to! Which is totally stupid?
# * picks can call channel types or separate channel names like: picks=['grad'] or picks=['MEG0111']
# * alterbnatively u can specify peaks and/or flats values for types of channels like: peak={'mag':3.5e-11, 'grad': 4e-11} (but not the names of channels).
# * min_duration is in minutes
# * peaks and flats are in the same scale as the data, like: 1e-13

# Differences with my function in Funks.Peaks_meg_gc.ipynb: 
# - no option to set distance between up and down peak to be concedered a pair. They calculate peak like: abs(a[i+1] - a[i]) ≥ peak (from docs).
#   So we can probably adjust indexing of 'a' here to decide which peaks concider as an up+down pair.
# - This function need raw obj as input. So not epochs and not just a piece of data -> need to adjust it for epochs. 
# - Can set here extra: flat, percent of bad, min duration and actual height of peaks and flats.

# Now need to add these annotation obj to the raw:
raw_cropped.set_annotations(raw_cropped.annotations + amplit_annot[0])  
raw_cropped.info['bads'] = amplit_annot[1]
# annotate_amplitude creates a tuple, not just annotations, like other functions from this series. 
# Second part of tuple is 'bads' (whole channel is bad, too many bad segments). 
# This whole tuple can't be added to annots in raw, need to slice the first part like: [0]


print('\nAnnotations added to raw:', raw_cropped.annotations)
print('Channels added to bads in raw:', raw_cropped.info['bads'],"\n")


# ANNOTATIONS ARE SUMMED HERE. SO IF YOU WANT TO OVERWRITE THEM, RUN ALL CELLS ABOVE AGAIN OR JUST DELETE THE ANNOTATIONS like: 
# idx=[a for a in range(0,len(raw_cropped.annotations))] #all annotations chosen for removal
# raw_cropped.annotations.delete(idx)
# ONLY RERUNNING THIS ONE CELL WILL NOT REMOVE OLD ANNOTS FROM RAW. 

raw_cropped.plot() #plot the data with annotations

#export annotation to dataframe:
df_annot=raw_cropped.annotations.to_data_frame()
df_annot


Finding segments below or above PTP threshold.

Annotations added to raw: <Annotations | 6 segments: BAD_flat (5), BAD_peak (1)>
Channels added to bads in raw: [] 



2022-07-19 22:02:41,199 - numexpr.utils - INFO - NumExpr defaulting to 8 threads.


Using qt as 2D backend.
Opening raw-browser...
Using pyopengl with version 3.1.1a1


2022-07-19 22:02:42,625 - OpenGL.acceleratesupport - INFO - No OpenGL_accelerate module loaded: No module named 'OpenGL_accelerate'


,onset,duration,description
0,2021-08-11 12:36:33.114942,0.005,BAD_peak
1,2021-08-11 12:37:00.189942,0.005,BAD_flat
2,2021-08-11 12:38:19.231942,0.005,BAD_flat
3,2021-08-11 12:38:52.072942,0.005,BAD_flat
4,2021-08-11 12:43:40.273942,0.005,BAD_flat
5,2021-08-11 12:45:14.584942,0.005,BAD_flat


In [4]:
# Since this function doesnt give channel names, need to loop over every channel and add its name into every annotation.
# mne.Annotations obj doesnt allow changing, so will have to create a new object and write explicitly all info + channel name there.

def get_amplitude_annots_per_channel(raw: mne.io.Raw, peak: float, flat: float, ch_type_names: list) -> tuple[mne.Annotations, list]:
    """Function creates amplitude (peak-to-peak annotations for every channel separately"""
    
    amplit_annot_with_ch_names=mne.Annotations(onset=[], duration=[], description=[], orig_time=raw.annotations.orig_time) #initialize 
    # check if we need to set orig_time. 
    # Docs: "If orig_time is None, the annotations are synced to the start of the data (0 seconds). 
    # Otherwise the annotations are synced to sample 0 and raw.first_samp is taken into account the same way as with events."

    bad_channels=[]

    for channel in ch_type_names:
        #get annotation object:
        amplit_annot=mne.preprocessing.annotate_amplitude(raw, peak=peak, flat=flat , bad_percent=5, min_duration=0.005, picks=[channel[0]], verbose=False)
        
        bad_channels.append(amplit_annot[1]) #Can later add these into annotation as well.

        if len(amplit_annot[0])>0:

            #create new annot obj and add there all data + channel name:
            amplit_annot_with_ch_names.append(onset=amplit_annot[0][0]['onset'], duration=amplit_annot[0][0]['duration'], description=amplit_annot[0][0]['description'], ch_names=[[channel[0]]])


    return amplit_annot_with_ch_names, bad_channels



In [5]:
# # clear annotations:
# idx=[a for a in range(0,len(raw_cropped.annotations))] #all annotations chosen for removal
# raw_cropped.annotations.delete(idx)

peak=3.5e-11 
flat=3e-15

amplit_annot_with_ch_names_mags, bad_channels=get_amplitude_annots_per_channel(raw_cropped, peak, flat, ch_type_names=mags)
#amplit_annot_with_ch_names_grads=get_amplitude_annots_per_channel(raw_cropped, peak, flat, ch_type_names=grads)

#extract annots to data frame:
df_annot=amplit_annot_with_ch_names_mags.to_data_frame()
df_annot


,onset,duration,description,ch_names
0,2021-08-11 12:37:00.189942,0.005,BAD_flat,"(MEG0231,)"
1,2021-08-11 12:38:19.231942,0.005,BAD_flat,"(MEG0411,)"
2,2021-08-11 12:38:52.072942,0.005,BAD_flat,"(MEG1611,)"
3,2021-08-11 12:43:40.273942,0.005,BAD_flat,"(MEG0421,)"
4,2021-08-11 12:45:14.584942,0.005,BAD_flat,"(MEG0641,)"


In [6]:
# add annotations to raw obj:

raw_cropped.set_annotations(raw_cropped.annotations + amplit_annot_with_ch_names_mags)  

#plot the data with annotations
raw_cropped.plot() 

raw_cropped.annotations

#Adding annotations to raw should work like this, but for some reason it doesnt? It did in the loop, when I add them one dy one.

Opening raw-browser...
Using pyopengl with version 3.1.1a1


<Annotations | 11 segments, channel-specific: BAD_flat (10), BAD_peak (1)>

In [7]:
# Annotate breaks - doesnt work..

break_annots = mne.preprocessing.annotate_break(
    raw=raw_cropped,
    min_break_duration=20,  # consider segments of at least 20 s duration
    t_start_after_previous=5,  # start annotation 5 s after end of previous one
    t_stop_before_next=2  # stop annotation 2 s before beginning of next one
)


raw_cropped.set_annotations(raw_cropped.annotations + break_annots)  # add to existing
raw_cropped.plot()

Ignoring annotations with descriptions starting with: bad, edge


IndexError: list index out of range

In [171]:
# Annotate movement - unfinished. need to find head positiins first.

chpi_freqs, ch_idx, chpi_codes = mne.chpi.get_chpi_info(info=raw.info)
print(f'cHPI coil frequencies extracted from raw: {chpi_freqs} Hz')

#chpi_amplitudes = mne.chpi.compute_chpi_amplitudes(raw)

chpi_locs = mne.chpi.extract_chpi_locs_ctf(raw)
head_pos = mne.chpi.compute_head_pos(raw.info, chpi_locs)

Using 5 HPI coils: 261 278 294 311 328 Hz
cHPI coil frequencies extracted from raw: [261.33300781 278.         294.66699219 311.33300781 328.        ] Hz


RuntimeError: Could not find all 9 cHPI channels

In [14]:
raw_cropped.info

<Info | 24 non-empty values
 acq_pars: ACQactiveGround 1 ACQch.BIO001.gain 2000 ACQch.BIO001.highpass ...
 bads: []
 ch_names: IASX+, IASX-, IASY+, IASY-, IASZ+, IASZ-, IAS_DX, IAS_DY, IAS_X, ...
 chs: 11 IAS, 102 Magnetometers, 204 Gradiometers, 1 Stimulus, 1 SYST
 custom_ref_applied: False
 description: TRIUX system at Oldenburg_3127
 dev_head_t: MEG device -> head transform
 dig: 356 items (3 Cardinal, 5 HPI, 348 Extra)
 events: 1 item (list)
 experimenter: Meg User (meg)
 file_id: 4 items (dict)
 gantry_angle: 68.0
 highpass: 0.1 Hz
 hpi_meas: 1 item (list)
 hpi_results: 1 item (list)
 hpi_subsystem: 3 items (dict)
 line_freq: 50.0
 lowpass: 330.0 Hz
 meas_date: 2021-08-11 12:05:17 UTC
 meas_id: 4 items (dict)
 nchan: 319
 proj_id: 1 item (ndarray)
 proj_name: mikado
 projs: magn8_iasoff_68deg.fif : PCA-v1: off, magn8_iasoff_68deg.fif : ...
 sfreq: 1000.0 Hz
 subject_info: 6 items (dict)
>

In [ ]:

mne.preprocessing.annotate_movement(raw_cropped, pos, rotation_velocity_limit=None, translation_velocity_limit=None, mean_distance_limit=None, use_dev_head_trans='average')

# pos: The position and quaternion parameters from cHPI fitting. Obtained with mne.chpi functions.
# https://mne.tools/stable/generated/mne.chpi.compute_head_pos.html#mne.chpi.compute_head_pos




In [ ]:
# https://pyprep.readthedocs.io/en/latest/generated/pyprep.NoisyChannels.html#pyprep.NoisyChannels

# This class implements all of the noisy channel detection methods used in the PREP pipeline, as described in:
# Bigdely-Shamlo, N., Mullen, T., Kothe, C., Su, K. M., Robbins, K. A. (2015). The PREP pipeline: 
# standardized preprocessing for large-scale EEG analysis. Frontiers in Neuroinformatics, 9, 16.

noisy = pp.NoisyChannels(raw) #first, add raw to class, then all the noisy channels finding performed on this new object.

# Doesnt work. Only EEG is supported. Tried to change channel type in just to try in:
# opt/anaconda3/envs/mne_new/lib/python3.9/site-packages/pyprep/find_noisy_channels.py
# to self.raw_mne.pick_types(meg=True, verbose=True)
# no luck, it gives traceback on this line. 
# (If u explicitely call raw.pick_types(meg=True, verbose=True) - everything works fine - see cell above).

# Dont know where the issue is exactly, but in the end we ll need to rewrite this whole function 
# to work with MEG if we really need it.

from time import perf_counter

start_time = perf_counter()
#noisy.find_bad_by_ransac(channel_wise=True)
noisy.find_all_bads(ransac=True, channel_wise=False, max_chunk_size=None)
print("--- %s seconds ---" % (perf_counter() - start_time))